In [1]:
import pandas as pd
import pyarrow as pa
import json

In [40]:
#tr_targets = pd.read_csv('train_targets.csv')
all_events = pd.read_csv("test_events.csv")
video = pd.read_csv("video_info_v3.csv")
all_events

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid
0,2024-06-01 15:26:44+03:00,Udmurtiya Republic,desktop,browser,Windows,Yandex Browser,2593,video_212730,22206
1,2024-06-01 12:25:29+03:00,Adygeya Republic,smartphone,mobile app,Android,Rutube,960,video_235114,34531
2,2024-06-01 17:23:12+03:00,Astrakhan Oblast,smartphone,mobile app,Android,Rutube,4695,video_26520,25830
3,2024-06-01 15:37:37+03:00,Khakasiya Republic,smartphone,browser,Android,Chrome Mobile,2490,video_465561,14838
4,2024-06-01 21:30:11+03:00,Moscow,smartphone,mobile app,Android,Rutube,1117,video_102934,13718
...,...,...,...,...,...,...,...,...,...
587730,2024-06-30 17:33:36+03:00,Krasnodar Krai,smartphone,mobile app,Android,Rutube,2509,video_243869,23170
587731,2024-06-30 12:36:00+03:00,Moscow,tablet,browser,Android,Firefox Mobile,555,video_193920,43241
587732,2024-06-30 06:55:31+03:00,Leningradskaya Oblast',desktop,browser,Windows,Chrome,1713,video_246870,31204
587733,2024-06-30 00:16:31+03:00,Saratov Oblast,smartphone,browser,Android,Chrome Mobile,51,video_15940,40889


In [41]:
train = all_events

In [42]:
train['region'] = train['region'].replace(r"[’'`]", '', regex=True)

In [43]:
train = train.merge(video, how='inner', on='rutube_video_id')

In [44]:
train

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,title,category,duration,author_id,label
0,2024-06-01 15:26:44+03:00,Udmurtiya Republic,desktop,browser,Windows,Yandex Browser,2593,video_212730,22206,Отчаянные Домохозяйки 7 сезон 21 серия,Сериалы,2610785,1089828,14
1,2024-06-01 12:25:29+03:00,Adygeya Republic,smartphone,mobile app,Android,Rutube,960,video_235114,34531,Вот как воевали США во Второй мировой войне!,Разное,1080320,1009406,30
2,2024-06-01 17:23:12+03:00,Astrakhan Oblast,smartphone,mobile app,Android,Rutube,4695,video_26520,25830,"Бесподобный мистер Фокс (мультфильм, 2009)",Фильмы,5208416,1090779,30
3,2024-06-01 15:37:37+03:00,Khakasiya Republic,smartphone,browser,Android,Chrome Mobile,2490,video_465561,14838,2- Вышивальные планы на 2024г,Хобби,2556011,1017105,30
4,2024-06-01 21:30:11+03:00,Moscow,smartphone,mobile app,Android,Rutube,1117,video_102934,13718,"Суперниндзя. Дети, 3 выпуск",Телепередачи,8061640,1009210,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587730,2024-06-30 17:33:36+03:00,Krasnodar Krai,smartphone,mobile app,Android,Rutube,2509,video_243869,23170,"ТАК СКАЗАТЬ: Фидаров – о феминизме, поствоенно...",Интервью,3524374,1022719,30
587731,2024-06-30 12:36:00+03:00,Moscow,tablet,browser,Android,Firefox Mobile,555,video_193920,43241,Топ-5 Игр На Телефон 2024!,Видеоигры,220550,1018241,11
587732,2024-06-30 06:55:31+03:00,Leningradskaya Oblast,desktop,browser,Windows,Chrome,1713,video_246870,31204,"Творческая встреча с известным писателем, публ...",Интервью,2855611,1010000,30
587733,2024-06-30 00:16:31+03:00,Saratov Oblast,smartphone,browser,Android,Chrome Mobile,51,video_15940,40889,СЕРИАЛЫ 2024 | ПОДБОРКА | SUDARIKOVA KATE,Сериалы,247501,1012086,11


In [45]:
from datetime import timedelta

train['event_timestamp'] = pd.to_datetime(train['event_timestamp'])
train

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,title,category,duration,author_id,label
0,2024-06-01 15:26:44+03:00,Udmurtiya Republic,desktop,browser,Windows,Yandex Browser,2593,video_212730,22206,Отчаянные Домохозяйки 7 сезон 21 серия,Сериалы,2610785,1089828,14
1,2024-06-01 12:25:29+03:00,Adygeya Republic,smartphone,mobile app,Android,Rutube,960,video_235114,34531,Вот как воевали США во Второй мировой войне!,Разное,1080320,1009406,30
2,2024-06-01 17:23:12+03:00,Astrakhan Oblast,smartphone,mobile app,Android,Rutube,4695,video_26520,25830,"Бесподобный мистер Фокс (мультфильм, 2009)",Фильмы,5208416,1090779,30
3,2024-06-01 15:37:37+03:00,Khakasiya Republic,smartphone,browser,Android,Chrome Mobile,2490,video_465561,14838,2- Вышивальные планы на 2024г,Хобби,2556011,1017105,30
4,2024-06-01 21:30:11+03:00,Moscow,smartphone,mobile app,Android,Rutube,1117,video_102934,13718,"Суперниндзя. Дети, 3 выпуск",Телепередачи,8061640,1009210,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587730,2024-06-30 17:33:36+03:00,Krasnodar Krai,smartphone,mobile app,Android,Rutube,2509,video_243869,23170,"ТАК СКАЗАТЬ: Фидаров – о феминизме, поствоенно...",Интервью,3524374,1022719,30
587731,2024-06-30 12:36:00+03:00,Moscow,tablet,browser,Android,Firefox Mobile,555,video_193920,43241,Топ-5 Игр На Телефон 2024!,Видеоигры,220550,1018241,11
587732,2024-06-30 06:55:31+03:00,Leningradskaya Oblast,desktop,browser,Windows,Chrome,1713,video_246870,31204,"Творческая встреча с известным писателем, публ...",Интервью,2855611,1010000,30
587733,2024-06-30 00:16:31+03:00,Saratov Oblast,smartphone,browser,Android,Chrome Mobile,51,video_15940,40889,СЕРИАЛЫ 2024 | ПОДБОРКА | SUDARIKOVA KATE,Сериалы,247501,1012086,11


In [46]:
train['event_timestamp'] = train['event_timestamp'] - timedelta(hours=3)

In [47]:
time_zones = {
    'Tatarstan Republic': 3,
    'Bashkortostan Republic': 5,
    'Novosibirsk Oblast': 7,
    'Moscow': 3,
    'Moscow Oblast': 3,
    'Omsk Oblast': 7,
    'Chuvashia': 3,
    'Krasnoyarsk Krai': 7,
    'Chelyabinsk': 5,
    'Kamchatka': 11,
    'Nizhny Novgorod Oblast': 3,
    'Krasnodar Krai': 3,
    'Volgograd Oblast': 3,
    'Kaliningrad Oblast': 2,
    'Kuzbass': 7,
    'Stavropol Kray': 3,
    'Samara Oblast': 4,
    'Amur Oblast': 9,
    'Sverdlovsk Oblast': 5,
    'St.-Petersburg': 3,
    'Yamalo-Nenets': 5,
    'Orenburg Oblast': 5,
    'Khanty-Mansia': 5,
    'Kaluga Oblast': 3,
    'Tomsk Oblast': 7,
    'Novgorod Oblast': 3,
    'Arkhangelskaya': 3,
    'North Ossetia–Alania': 3,
    'Kursk Oblast': 3,
    "Leningradskaya Oblast'": 3,
    'Krasnoyarskiy': 7,
    'Ivanovo Oblast': 3,
    'Altay Kray': 7,
    'Kurgan Oblast': 5,
    'Kostroma Oblast': 3,
    'Bryansk Oblast': 3,
    'Dagestan': 3,
    'Lipetsk Oblast': 3,
    'Vladimir Oblast': 3,
    'Kirov Oblast': 3,
    'Khabarovsk': 9,
    'Tambov Oblast': 3,
    'Chukotka': 12,
    'Voronezh Oblast': 3,
    'Sverdlovsk': 5,
    'Tula Oblast': 3,
    'Krasnodarskiy': 3,
    'Irkutsk Oblast': 8,
    'Saratov Oblast': 4,
    'Khakasiya Republic': 7,
    'Penza': 3,
    'Perm Krai': 5,
    'Oryol oblast': 3,
    'Vladimir': 3,
    'Smolensk Oblast': 3,
    'Penza Oblast': 3,
    'Mordoviya Republic': 3,
    'Tyumen’ Oblast': 5,
    'Sakha': 9,
    'Primorye': 10,
    'Zabaykalskiy (Transbaikal) Kray': 9,
    'Vologda Oblast': 3,
    'Yaroslavl Oblast': 3,
    'Crimea': 3,
    'Rostov': 3,
    'Ryazan Oblast': 3,
    'Perm': 5,
    'Chechnya': 3,
    'Udmurtiya Republic': 3,
    'Tver Oblast': 3,
    'Buryatiya Republic': 8,
    'Belgorod Oblast': 3,
    'Kaluga': 3,
    'Astrakhan Oblast': 3,
    'Karelia': 3,
    'Murmansk': 3,
    'Adygeya Republic': 3,
    'Kemerovo Oblast': 7,
    'Mariy-El Republic': 3,
    'Kursk': 3,
    'Saratovskaya Oblast': 4,
    'Sakhalin Oblast': 10,
    'Ivanovo': 3,
    'Tyumen Oblast': 5,
    'Stavropol’ Kray': 3,
    'Voronezj': 3,
    'Karachayevo-Cherkesiya Republic': 3,
    'Kabardino-Balkariya Republic': 3,
    'Ulyanovsk': 4,
    'North Ossetia': 3,
    'Komi': 3,
    'Smolensk': 3,
    'Tver’ Oblast': 3,
    'Sebastopol City': 3,
    'Pskov Oblast': 3,
    'Tula': 3,
    'Orel Oblast': 3,
    'Jaroslavl': 3,
    'Tambov': 3,
    'Kalmykiya Republic': 3,
    'Primorskiy (Maritime) Kray': 10,
    'Altai': 7,
    'Magadan Oblast': 10,
    'Vologda': 3,
    'Tyva Republic': 7,
    'Nenets': 3,
    'Smolenskaya Oblast': 3,
    'Jewish Autonomous Oblast': 10,
    'Astrakhan': 3,
    'Ingushetiya Republic': 3,
    'Kirov': 3,
    'Transbaikal Territory': 9,
    'Omsk': 7,
    'Kaliningrad': 2,
    'Stavropol Krai': 3,
    'Arkhangelsk Oblast': 3,
    'Leningradskaya Oblast': 3
}

In [48]:
train['timezone_offset'] = train['region'].map(time_zones)

In [49]:
train['event_timestamp_timezone'] = train['event_timestamp'] + pd.to_timedelta(train['timezone_offset'], unit='h')

In [50]:
train['morning'] = 0
train['day'] = 0
train['evening'] = 0
train['night'] = 0
train.loc[train['event_timestamp_timezone'].dt.hour.between(6, 12, inclusive='left'), 'morning'] = 1
train.loc[train['event_timestamp_timezone'].dt.hour.between(13, 18, inclusive='left'), 'day'] = 1
train.loc[train['event_timestamp_timezone'].dt.hour.between(19, 23, inclusive='left'), 'evening'] = 1
train.loc[train['event_timestamp_timezone'].dt.hour.between(0, 5, inclusive='both'), 'night'] = 1

In [51]:
with open("category.json", 'r') as f:
    cat_dict = json.loads(f.read())
train['category'] = train['category'].map(cat_dict)

In [52]:
with open("regions.json", 'r') as file:
    region_dict = json.loads(file.read())
train['region'] = train['region'].map(region_dict)
train

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,title,category,duration,author_id,label,timezone_offset,event_timestamp_timezone,morning,day,evening,night
0,2024-06-01 12:26:44+03:00,68,desktop,browser,Windows,Yandex Browser,2593,video_212730,22206,Отчаянные Домохозяйки 7 сезон 21 серия,1,2610785,1089828,14,3,2024-06-01 15:26:44+03:00,0,1,0,0
1,2024-06-01 09:25:29+03:00,76,smartphone,mobile app,Android,Rutube,960,video_235114,34531,Вот как воевали США во Второй мировой войне!,4,1080320,1009406,30,3,2024-06-01 12:25:29+03:00,0,0,0,0
2,2024-06-01 14:23:12+03:00,73,smartphone,mobile app,Android,Rutube,4695,video_26520,25830,"Бесподобный мистер Фокс (мультфильм, 2009)",2,5208416,1090779,30,3,2024-06-01 17:23:12+03:00,0,1,0,0
3,2024-06-01 12:37:37+03:00,49,smartphone,browser,Android,Chrome Mobile,2490,video_465561,14838,2- Вышивальные планы на 2024г,0,2556011,1017105,30,7,2024-06-01 19:37:37+03:00,0,0,1,0
4,2024-06-01 18:30:11+03:00,3,smartphone,mobile app,Android,Rutube,1117,video_102934,13718,"Суперниндзя. Дети, 3 выпуск",16,8061640,1009210,31,3,2024-06-01 21:30:11+03:00,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587730,2024-06-30 14:33:36+03:00,11,smartphone,mobile app,Android,Rutube,2509,video_243869,23170,"ТАК СКАЗАТЬ: Фидаров – о феминизме, поствоенно...",17,3524374,1022719,30,3,2024-06-30 17:33:36+03:00,0,1,0,0
587731,2024-06-30 09:36:00+03:00,3,tablet,browser,Android,Firefox Mobile,555,video_193920,43241,Топ-5 Игр На Телефон 2024!,11,220550,1018241,11,3,2024-06-30 12:36:00+03:00,0,0,0,0
587732,2024-06-30 03:55:31+03:00,29,desktop,browser,Windows,Chrome,1713,video_246870,31204,"Творческая встреча с известным писателем, публ...",17,2855611,1010000,30,3,2024-06-30 06:55:31+03:00,1,0,0,0
587733,2024-06-29 21:16:31+03:00,48,smartphone,browser,Android,Chrome Mobile,51,video_15940,40889,СЕРИАЛЫ 2024 | ПОДБОРКА | SUDARIKOVA KATE,1,247501,1012086,11,4,2024-06-30 01:16:31+03:00,0,0,0,1


In [53]:
train['duration'] = train['duration'] / 1000
train['percent'] = train['total_watchtime'] / train['duration']

In [54]:
train

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,title,...,duration,author_id,label,timezone_offset,event_timestamp_timezone,morning,day,evening,night,percent
0,2024-06-01 12:26:44+03:00,68,desktop,browser,Windows,Yandex Browser,2593,video_212730,22206,Отчаянные Домохозяйки 7 сезон 21 серия,...,2610.785,1089828,14,3,2024-06-01 15:26:44+03:00,0,1,0,0,0.993188
1,2024-06-01 09:25:29+03:00,76,smartphone,mobile app,Android,Rutube,960,video_235114,34531,Вот как воевали США во Второй мировой войне!,...,1080.320,1009406,30,3,2024-06-01 12:25:29+03:00,0,0,0,0,0.888626
2,2024-06-01 14:23:12+03:00,73,smartphone,mobile app,Android,Rutube,4695,video_26520,25830,"Бесподобный мистер Фокс (мультфильм, 2009)",...,5208.416,1090779,30,3,2024-06-01 17:23:12+03:00,0,1,0,0,0.901426
3,2024-06-01 12:37:37+03:00,49,smartphone,browser,Android,Chrome Mobile,2490,video_465561,14838,2- Вышивальные планы на 2024г,...,2556.011,1017105,30,7,2024-06-01 19:37:37+03:00,0,0,1,0,0.974174
4,2024-06-01 18:30:11+03:00,3,smartphone,mobile app,Android,Rutube,1117,video_102934,13718,"Суперниндзя. Дети, 3 выпуск",...,8061.640,1009210,31,3,2024-06-01 21:30:11+03:00,0,0,1,0,0.138557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587730,2024-06-30 14:33:36+03:00,11,smartphone,mobile app,Android,Rutube,2509,video_243869,23170,"ТАК СКАЗАТЬ: Фидаров – о феминизме, поствоенно...",...,3524.374,1022719,30,3,2024-06-30 17:33:36+03:00,0,1,0,0,0.711899
587731,2024-06-30 09:36:00+03:00,3,tablet,browser,Android,Firefox Mobile,555,video_193920,43241,Топ-5 Игр На Телефон 2024!,...,220.550,1018241,11,3,2024-06-30 12:36:00+03:00,0,0,0,0,2.516436
587732,2024-06-30 03:55:31+03:00,29,desktop,browser,Windows,Chrome,1713,video_246870,31204,"Творческая встреча с известным писателем, публ...",...,2855.611,1010000,30,3,2024-06-30 06:55:31+03:00,1,0,0,0,0.599872
587733,2024-06-29 21:16:31+03:00,48,smartphone,browser,Android,Chrome Mobile,51,video_15940,40889,СЕРИАЛЫ 2024 | ПОДБОРКА | SUDARIKOVA KATE,...,247.501,1012086,11,4,2024-06-30 01:16:31+03:00,0,0,0,1,0.206060


In [55]:
train = train.drop(['title'], axis=1)

In [56]:
mn = train['ua_client_type'].value_counts().idxmax()
train['ua_client_type'] = train['ua_client_type'].fillna(mn)

In [57]:
with open("client_types.json", 'r') as file:
    client_type_dict = json.loads(file.read())
train['ua_client_type'] = train['ua_client_type'].map(client_type_dict)

In [58]:
train

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,category,duration,author_id,label,timezone_offset,event_timestamp_timezone,morning,day,evening,night,percent
0,2024-06-01 12:26:44+03:00,68,desktop,0,Windows,Yandex Browser,2593,video_212730,22206,1,2610.785,1089828,14,3,2024-06-01 15:26:44+03:00,0,1,0,0,0.993188
1,2024-06-01 09:25:29+03:00,76,smartphone,1,Android,Rutube,960,video_235114,34531,4,1080.320,1009406,30,3,2024-06-01 12:25:29+03:00,0,0,0,0,0.888626
2,2024-06-01 14:23:12+03:00,73,smartphone,1,Android,Rutube,4695,video_26520,25830,2,5208.416,1090779,30,3,2024-06-01 17:23:12+03:00,0,1,0,0,0.901426
3,2024-06-01 12:37:37+03:00,49,smartphone,0,Android,Chrome Mobile,2490,video_465561,14838,0,2556.011,1017105,30,7,2024-06-01 19:37:37+03:00,0,0,1,0,0.974174
4,2024-06-01 18:30:11+03:00,3,smartphone,1,Android,Rutube,1117,video_102934,13718,16,8061.640,1009210,31,3,2024-06-01 21:30:11+03:00,0,0,1,0,0.138557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587730,2024-06-30 14:33:36+03:00,11,smartphone,1,Android,Rutube,2509,video_243869,23170,17,3524.374,1022719,30,3,2024-06-30 17:33:36+03:00,0,1,0,0,0.711899
587731,2024-06-30 09:36:00+03:00,3,tablet,0,Android,Firefox Mobile,555,video_193920,43241,11,220.550,1018241,11,3,2024-06-30 12:36:00+03:00,0,0,0,0,2.516436
587732,2024-06-30 03:55:31+03:00,29,desktop,0,Windows,Chrome,1713,video_246870,31204,17,2855.611,1010000,30,3,2024-06-30 06:55:31+03:00,1,0,0,0,0.599872
587733,2024-06-29 21:16:31+03:00,48,smartphone,0,Android,Chrome Mobile,51,video_15940,40889,1,247.501,1012086,11,4,2024-06-30 01:16:31+03:00,0,0,0,1,0.206060


In [59]:
mn = train['ua_os'].value_counts().idxmax()
train['ua_os'] = train['ua_os'].fillna(mn)

In [60]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['ua_os'] = le.fit_transform(train['ua_os'])

In [61]:
train

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,category,duration,author_id,label,timezone_offset,event_timestamp_timezone,morning,day,evening,night,percent
0,2024-06-01 12:26:44+03:00,68,desktop,0,16,Yandex Browser,2593,video_212730,22206,1,2610.785,1089828,14,3,2024-06-01 15:26:44+03:00,0,1,0,0,0.993188
1,2024-06-01 09:25:29+03:00,76,smartphone,1,0,Rutube,960,video_235114,34531,4,1080.320,1009406,30,3,2024-06-01 12:25:29+03:00,0,0,0,0,0.888626
2,2024-06-01 14:23:12+03:00,73,smartphone,1,0,Rutube,4695,video_26520,25830,2,5208.416,1090779,30,3,2024-06-01 17:23:12+03:00,0,1,0,0,0.901426
3,2024-06-01 12:37:37+03:00,49,smartphone,0,0,Chrome Mobile,2490,video_465561,14838,0,2556.011,1017105,30,7,2024-06-01 19:37:37+03:00,0,0,1,0,0.974174
4,2024-06-01 18:30:11+03:00,3,smartphone,1,0,Rutube,1117,video_102934,13718,16,8061.640,1009210,31,3,2024-06-01 21:30:11+03:00,0,0,1,0,0.138557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587730,2024-06-30 14:33:36+03:00,11,smartphone,1,0,Rutube,2509,video_243869,23170,17,3524.374,1022719,30,3,2024-06-30 17:33:36+03:00,0,1,0,0,0.711899
587731,2024-06-30 09:36:00+03:00,3,tablet,0,0,Firefox Mobile,555,video_193920,43241,11,220.550,1018241,11,3,2024-06-30 12:36:00+03:00,0,0,0,0,2.516436
587732,2024-06-30 03:55:31+03:00,29,desktop,0,16,Chrome,1713,video_246870,31204,17,2855.611,1010000,30,3,2024-06-30 06:55:31+03:00,1,0,0,0,0.599872
587733,2024-06-29 21:16:31+03:00,48,smartphone,0,0,Chrome Mobile,51,video_15940,40889,1,247.501,1012086,11,4,2024-06-30 01:16:31+03:00,0,0,0,1,0.206060


In [62]:
mn = train['ua_client_name'].value_counts().idxmax()
train['ua_client_name'] = train['ua_client_name'].fillna(mn)

In [63]:
with open("client_names.json", 'r') as file:
    client_name_dict = json.loads(file.read())
train['ua_client_name'] = train['ua_client_name'].map(client_name_dict)
train

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,category,duration,author_id,label,timezone_offset,event_timestamp_timezone,morning,day,evening,night,percent
0,2024-06-01 12:26:44+03:00,68,desktop,0,16,2,2593,video_212730,22206,1,2610.785,1089828,14,3,2024-06-01 15:26:44+03:00,0,1,0,0,0.993188
1,2024-06-01 09:25:29+03:00,76,smartphone,1,0,1,960,video_235114,34531,4,1080.320,1009406,30,3,2024-06-01 12:25:29+03:00,0,0,0,0,0.888626
2,2024-06-01 14:23:12+03:00,73,smartphone,1,0,1,4695,video_26520,25830,2,5208.416,1090779,30,3,2024-06-01 17:23:12+03:00,0,1,0,0,0.901426
3,2024-06-01 12:37:37+03:00,49,smartphone,0,0,0,2490,video_465561,14838,0,2556.011,1017105,30,7,2024-06-01 19:37:37+03:00,0,0,1,0,0.974174
4,2024-06-01 18:30:11+03:00,3,smartphone,1,0,1,1117,video_102934,13718,16,8061.640,1009210,31,3,2024-06-01 21:30:11+03:00,0,0,1,0,0.138557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587730,2024-06-30 14:33:36+03:00,11,smartphone,1,0,1,2509,video_243869,23170,17,3524.374,1022719,30,3,2024-06-30 17:33:36+03:00,0,1,0,0,0.711899
587731,2024-06-30 09:36:00+03:00,3,tablet,0,0,9,555,video_193920,43241,11,220.550,1018241,11,3,2024-06-30 12:36:00+03:00,0,0,0,0,2.516436
587732,2024-06-30 03:55:31+03:00,29,desktop,0,16,3,1713,video_246870,31204,17,2855.611,1010000,30,3,2024-06-30 06:55:31+03:00,1,0,0,0,0.599872
587733,2024-06-29 21:16:31+03:00,48,smartphone,0,0,0,51,video_15940,40889,1,247.501,1012086,11,4,2024-06-30 01:16:31+03:00,0,0,0,1,0.206060


In [64]:
with open("devices.json", 'r') as file:
    device_dict = json.loads(file.read())
train['ua_device_type'] = train['ua_device_type'].map(device_dict)

In [65]:
train

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,category,duration,author_id,label,timezone_offset,event_timestamp_timezone,morning,day,evening,night,percent
0,2024-06-01 12:26:44+03:00,68,0,0,16,2,2593,video_212730,22206,1,2610.785,1089828,14,3,2024-06-01 15:26:44+03:00,0,1,0,0,0.993188
1,2024-06-01 09:25:29+03:00,76,1,1,0,1,960,video_235114,34531,4,1080.320,1009406,30,3,2024-06-01 12:25:29+03:00,0,0,0,0,0.888626
2,2024-06-01 14:23:12+03:00,73,1,1,0,1,4695,video_26520,25830,2,5208.416,1090779,30,3,2024-06-01 17:23:12+03:00,0,1,0,0,0.901426
3,2024-06-01 12:37:37+03:00,49,1,0,0,0,2490,video_465561,14838,0,2556.011,1017105,30,7,2024-06-01 19:37:37+03:00,0,0,1,0,0.974174
4,2024-06-01 18:30:11+03:00,3,1,1,0,1,1117,video_102934,13718,16,8061.640,1009210,31,3,2024-06-01 21:30:11+03:00,0,0,1,0,0.138557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587730,2024-06-30 14:33:36+03:00,11,1,1,0,1,2509,video_243869,23170,17,3524.374,1022719,30,3,2024-06-30 17:33:36+03:00,0,1,0,0,0.711899
587731,2024-06-30 09:36:00+03:00,3,2,0,0,9,555,video_193920,43241,11,220.550,1018241,11,3,2024-06-30 12:36:00+03:00,0,0,0,0,2.516436
587732,2024-06-30 03:55:31+03:00,29,0,0,16,3,1713,video_246870,31204,17,2855.611,1010000,30,3,2024-06-30 06:55:31+03:00,1,0,0,0,0.599872
587733,2024-06-29 21:16:31+03:00,48,1,0,0,0,51,video_15940,40889,1,247.501,1012086,11,4,2024-06-30 01:16:31+03:00,0,0,0,1,0.206060


In [66]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10, max_iter=1000).fit(train[['ua_device_type', 'ua_client_type', 'ua_os', 'ua_client_name']])

In [100]:
train['device'] = kmeans.labels_
train['viewer_uid2'] = train['viewer_uid'] 

In [141]:
a = train['viewer_uid'].unique().tolist()
sorted(a)[-1]

60003

In [102]:
%%time
def get_mode(series):
    return series.mode().iloc[0] if not series.mode().empty else None

aggregated = train.groupby('viewer_uid').agg({
    'event_timestamp_timezone': ['count', 'min', 'max', 'mean'],
    'region': get_mode,
    'label': get_mode,
    'category': get_mode,
    'total_watchtime': ['mean', 'max', 'min', 'sum'],
    'device': get_mode,
    'author_id': get_mode,
    'rutube_video_id': get_mode,
    'percent': ['mean', 'min', 'max', 'sum'],
    'duration': ['min', 'max', 'sum', 'mean'],
    'viewer_uid2': get_mode
})
aggregated

CPU times: total: 57.2 s
Wall time: 57.8 s


event_timestamp_timezone                            \
                              count                       min   
viewer_uid                                                      
0                                 9 2024-06-08 22:18:01+03:00   
1                                 1 2024-06-05 10:30:05+03:00   
2                                 2 2024-06-01 19:25:07+03:00   
3                                 4 2024-06-01 11:06:29+03:00   
4                                52 2024-06-01 09:59:20+03:00   
...                             ...                       ...   
59999                             5 2024-06-30 21:35:01+03:00   
60000                             2 2024-06-30 23:04:48+03:00   
60001                             1 2024-06-30 22:21:04+03:00   
60002                             2 2024-06-30 22:23:54+03:00   
60003                             2 2024-06-30 22:36:48+03:00   

                                                                          \
                                 max                                mean   
viewer_uid                                                                 
0          2024-06-20 00:33:07+03:00           2024-06-13 07:57:25+03:00   
1          2024-06-05 10:30:05+03:00           2024-06-05 10:30:05+03:00   
2          2024-06-02 15:29:09+03:00           2024-06-02 05:27:08+03:00   
3          2024-06-29 13:48:09+03:00    2024-06-17 06:44:21.500000+03:00   
4          2024-06-30 22:23:37+03:00 2024-06-12 14:54:37.442307584+03:00   
...                              ...                                 ...   
59999      2024-06-30 23:15:38+03:00           2024-06-30 22:23:31+03:00   
60000      2024-06-30 23:29:56+03:00           2024-06-30 23:17:22+03:00   
60001      2024-06-30 22:21:04+03:00           2024-06-30 22:21:04+03:00   
60002      2024-06-30 23:59:04+03:00           2024-06-30 23:11:29+03:00   
60003      2024-06-30 23:42:07+03:00    2024-06-30 23:09:27.500000+03:00   

             region    label category total_watchtime              ...  \
           get_mode get_mode get_mode            mean   max   min  ...   
viewer_uid                                                         ...   
0                52       30       17     1602.111111  4821    60  ...   
1                78       30       17       92.000000    92    92  ...   
2                14       30       17     1735.000000  2560   910  ...   
3                11       40        5     1119.500000  1202  1082  ...   
4                14       35        1     1411.961538  4919    31  ...   
...             ...      ...      ...             ...   ...   ...  ...   
59999             3       30       16     1026.200000  3213    70  ...   
60000            23       30       13      403.500000   757    50  ...   
60001            19       30       10     1780.000000  1780  1780  ...   
60002            10       31       16     2566.500000  4941   192  ...   
60003            19        8        1     1649.000000  2706   592  ...   

           rutube_video_id   percent                                 duration  \
                  get_mode      mean       min       max        sum       min   
viewer_uid                                                                      
0             video_144865  0.482009  0.020788  0.997214   4.338079  1780.480   
1              video_95979  0.057440  0.057440  0.057440   0.057440  1601.680   
2             video_247624  0.481690  0.252645  0.710736   0.963381  3601.898   
3              video_11911  0.987800  0.950056  1.017071   3.951202  1100.075   
4             video_104628  0.526249  0.012095  0.983639  27.364966  1034.529   
...                    ...       ...       ...       ...        ...       ...   
59999         video_259886  0.268399  0.010210  0.925670   1.341996  3120.200   
60000         video_298886  0.143794  0.008911  0.278676   0.287587  2716.416   
60001          video_76818  4.674124  4.674124  4.674124   4.674124   380.820   
60002         video_393399

In [103]:
aggregated = aggregated.reset_index(drop=False)
aggregated

viewer_uid event_timestamp_timezone                            \
                                    count                       min   
0              0                        9 2024-06-08 22:18:01+03:00   
1              1                        1 2024-06-05 10:30:05+03:00   
2              2                        2 2024-06-01 19:25:07+03:00   
3              3                        4 2024-06-01 11:06:29+03:00   
4              4                       52 2024-06-01 09:59:20+03:00   
...          ...                      ...                       ...   
59999      59999                        5 2024-06-30 21:35:01+03:00   
60000      60000                        2 2024-06-30 23:04:48+03:00   
60001      60001                        1 2024-06-30 22:21:04+03:00   
60002      60002                        2 2024-06-30 22:23:54+03:00   
60003      60003                        2 2024-06-30 22:36:48+03:00   

                                                                      region  \
                            max                                mean get_mode   
0     2024-06-20 00:33:07+03:00           2024-06-13 07:57:25+03:00       52   
1     2024-06-05 10:30:05+03:00           2024-06-05 10:30:05+03:00       78   
2     2024-06-02 15:29:09+03:00           2024-06-02 05:27:08+03:00       14   
3     2024-06-29 13:48:09+03:00    2024-06-17 06:44:21.500000+03:00       11   
4     2024-06-30 22:23:37+03:00 2024-06-12 14:54:37.442307584+03:00       14   
...                         ...                                 ...      ...   
59999 2024-06-30 23:15:38+03:00           2024-06-30 22:23:31+03:00        3   
60000 2024-06-30 23:29:56+03:00           2024-06-30 23:17:22+03:00       23   
60001 2024-06-30 22:21:04+03:00           2024-06-30 22:21:04+03:00       19   
60002 2024-06-30 23:59:04+03:00           2024-06-30 23:11:29+03:00       10   
60003 2024-06-30 23:42:07+03:00    2024-06-30 23:09:27.500000+03:00       19   

         label category total_watchtime        ... rutube_video_id   percent  \
      get_mode get_mode            mean   max  ...        get_mode      mean   
0           30       17     1602.111111  4821  ...    video_144865  0.482009   
1           30       17       92.000000    92  ...     video_95979  0.057440   
2           30       17     1735.000000  2560  ...    video_247624  0.481690   
3           40        5     1119.500000  1202  ...     video_11911  0.987800   
4           35        1     1411.961538  4919  ...    video_104628  0.526249   
...        ...      ...             ...   ...  ...             ...       ...   
59999       30       16     1026.200000  3213  ...    video_259886  0.268399   
60000       30       13      403.500000   757  ...    video_298886  0.143794   
60001       30       10     1780.000000  1780  ...     video_76818  4.674124   
60002       31       16     2566.500000  4941  ...    video_393399  0.461189   
60003        8        1     1649.000000  2706  ...    video_462780  0.463527   

                                      duration                        \
            min       max        sum       min       max         sum   
0      0.020788  0.997214   4.338079  1780.480  5650.134   29170.703   
1      0.057440  0.057440   0.057440  1601.680  1601.680    1601.680   
2      0.252645  0.710736   0.963381  3601.898  3601.898    7203.796   
3      0.950056  1.017071   3.951202  1100.075  1181.825    4532.333   
4      0.012095  0.983639  27.364966  1034.529  7426.427  137341.970   
...         ...       ...        ...       ...       ...         ...   
59999  0.010210  0.925670   1.341996  3120.200  6856.000   22128.920   
60000  0.008911  0.278676   0.287587  2716.416  5610.840    8327.256   
60001  4.674124  4.674124   4.674124   380.820   380.820     380.820   
60002  0.033229  0.889149   0.922378  5557.000  5778.000   11335.000   
60003  0.167372  0.759682   0.927054  3537.025  3562.016    7099.041   

                   viewer_uid2  
              mean    get_mode

In [110]:
aggregated.columns = ['_'.join(filter(None, col)).strip() for col in aggregated.columns]
aggregated.columns

Index(['viewer_uid', 'event_timestamp_timezone_count',
       'event_timestamp_timezone_min', 'event_timestamp_timezone_max',
       'event_timestamp_timezone_mean', 'region_get_mode', 'label_get_mode',
       'category_get_mode', 'total_watchtime_mean', 'total_watchtime_max',
       'total_watchtime_min', 'total_watchtime_sum', 'device_get_mode',
       'author_id_get_mode', 'rutube_video_id_get_mode', 'percent_mean',
       'percent_min', 'percent_max', 'percent_sum', 'duration_min',
       'duration_max', 'duration_sum', 'duration_mean',
       'viewer_uid2_get_mode'],
      dtype='object')

In [111]:
aggregated['event_timestamp_timezone_min'] = aggregated['event_timestamp_timezone_min'].apply(lambda x: x.timestamp())
aggregated['event_timestamp_timezone_max'] = aggregated['event_timestamp_timezone_max'].apply(lambda x: x.timestamp())
aggregated['event_timestamp_timezone_mean'] = aggregated['event_timestamp_timezone_mean'].apply(lambda x: x.timestamp())
aggregated

,viewer_uid,event_timestamp_timezone_count,event_timestamp_timezone_min,event_timestamp_timezone_max,event_timestamp_timezone_mean,region_get_mode,label_get_mode,category_get_mode,total_watchtime_mean,total_watchtime_max,...,rutube_video_id_get_mode,percent_mean,percent_min,percent_max,percent_sum,duration_min,duration_max,duration_sum,duration_mean,viewer_uid2_get_mode
0,0,9,1.717874e+09,1.718833e+09,1.718255e+09,52,30,17,1602.111111,4821,...,video_144865,0.482009,0.020788,0.997214,4.338079,1780.480,5650.134,29170.703,3241.189222,0
1,1,1,1.717573e+09,1.717573e+09,1.717573e+09,78,30,17,92.000000,92,...,video_95979,0.057440,0.057440,0.057440,0.057440,1601.680,1601.680,1601.680,1601.680000,1
2,2,2,1.717259e+09,1.717331e+09,1.717295e+09,14,30,17,1735.000000,2560,...,video_247624,0.481690,0.252645,0.710736,0.963381,3601.898,3601.898,7203.796,3601.898000,2
3,3,4,1.717229e+09,1.719658e+09,1.718596e+09,11,40,5,1119.500000,1202,...,video_11911,0.987800,0.950056,1.017071,3.951202,1100.075,1181.825,4532.333,1133.083250,3
4,4,52,1.717225e+09,1.719775e+09,1.718193e+09,14,35,1,1411.961538,4919,...,video_104628,0.526249,0.012095,0.983639,27.364966,1034.529,7426.427,137341.970,2641.191731,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59999,59999,5,1.719773e+09,1.719779e+09,1.719775e+09,3,30,16,1026.200000,3213,...,video_259886,0.268399,0.010210,0.925670,1.341996,3120.200,6856.000,22128.920,4425.784000,59999
60000,60000,2,1.719778e+09,1.719779e+09,1.719779e+09,23,30,13,403.500000,757,...,video_298886,0.143794,0.008911,0.278676,0.287587,2716.416,5610.840,8327.256,4163.628000,60000
60001,60001,1,1.719775e+09,1.719775e+09,1.719775e+09,19,30,10,1780.000000,1780,...,video_76818,4.674124,4.674124,4.674124,4.674124,380.820,380.820,380.820,380.820000,60001
60002,60002,2,1.719775e+09,1.719781e+09,1.719778e+09,10,31,16,2566.500000,4941,...,video_393399,0.461189,0.033229,0.889149,0.922378,5557.000,5778.000,11335.000,5667.500000,60002


In [112]:
t = pa.Table.from_pandas(train)

In [113]:
time_sum = t.select(['viewer_uid', 'morning', 'day','evening', 'night']).group_by(['viewer_uid']).aggregate([('morning', "sum"), ('day', "sum"), ('evening', "sum"), ('night', "sum")])
time_sum = time_sum.to_pandas()

In [114]:
aggregated = aggregated.merge(time_sum, how='inner', on='viewer_uid')
aggregated

,viewer_uid,event_timestamp_timezone_count,event_timestamp_timezone_min,event_timestamp_timezone_max,event_timestamp_timezone_mean,region_get_mode,label_get_mode,category_get_mode,total_watchtime_mean,total_watchtime_max,...,percent_sum,duration_min,duration_max,duration_sum,duration_mean,viewer_uid2_get_mode,morning_sum,day_sum,evening_sum,night_sum
0,0,9,1.717874e+09,1.718833e+09,1.718255e+09,52,30,17,1602.111111,4821,...,4.338079,1780.480,5650.134,29170.703,3241.189222,0,0,3,2,3
1,1,1,1.717573e+09,1.717573e+09,1.717573e+09,78,30,17,92.000000,92,...,0.057440,1601.680,1601.680,1601.680,1601.680000,1,1,0,0,0
2,2,2,1.717259e+09,1.717331e+09,1.717295e+09,14,30,17,1735.000000,2560,...,0.963381,3601.898,3601.898,7203.796,3601.898000,2,0,1,1,0
3,3,4,1.717229e+09,1.719658e+09,1.718596e+09,11,40,5,1119.500000,1202,...,3.951202,1100.075,1181.825,4532.333,1133.083250,3,1,2,0,0
4,4,52,1.717225e+09,1.719775e+09,1.718193e+09,14,35,1,1411.961538,4919,...,27.364966,1034.529,7426.427,137341.970,2641.191731,4,2,11,30,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59999,59999,5,1.719773e+09,1.719779e+09,1.719775e+09,3,30,16,1026.200000,3213,...,1.341996,3120.200,6856.000,22128.920,4425.784000,59999,0,0,3,0
60000,60000,2,1.719778e+09,1.719779e+09,1.719779e+09,23,30,13,403.500000,757,...,0.287587,2716.416,5610.840,8327.256,4163.628000,60000,0,0,0,0
60001,60001,1,1.719775e+09,1.719775e+09,1.719775e+09,19,30,10,1780.000000,1780,...,4.674124,380.820,380.820,380.820,380.820000,60001,0,0,1,0
60002,60002,2,1.719775e+09,1.719781e+09,1.719778e+09,10,31,16,2566.500000,4941,...,0.922378,5557.000,5778.000,11335.000,5667.500000,60002,0,0,1,0


In [115]:
videos = video['rutube_video_id'].unique()
video_dict = {v: v_id for v_id, v in enumerate(videos)}

In [116]:
aggregated['rutube_video_id_get_mode'] =  aggregated['rutube_video_id_get_mode'].map(video_dict)

In [117]:
aggregated

,viewer_uid,event_timestamp_timezone_count,event_timestamp_timezone_min,event_timestamp_timezone_max,event_timestamp_timezone_mean,region_get_mode,label_get_mode,category_get_mode,total_watchtime_mean,total_watchtime_max,...,percent_sum,duration_min,duration_max,duration_sum,duration_mean,viewer_uid2_get_mode,morning_sum,day_sum,evening_sum,night_sum
0,0,9,1.717874e+09,1.718833e+09,1.718255e+09,52,30,17,1602.111111,4821,...,4.338079,1780.480,5650.134,29170.703,3241.189222,0,0,3,2,3
1,1,1,1.717573e+09,1.717573e+09,1.717573e+09,78,30,17,92.000000,92,...,0.057440,1601.680,1601.680,1601.680,1601.680000,1,1,0,0,0
2,2,2,1.717259e+09,1.717331e+09,1.717295e+09,14,30,17,1735.000000,2560,...,0.963381,3601.898,3601.898,7203.796,3601.898000,2,0,1,1,0
3,3,4,1.717229e+09,1.719658e+09,1.718596e+09,11,40,5,1119.500000,1202,...,3.951202,1100.075,1181.825,4532.333,1133.083250,3,1,2,0,0
4,4,52,1.717225e+09,1.719775e+09,1.718193e+09,14,35,1,1411.961538,4919,...,27.364966,1034.529,7426.427,137341.970,2641.191731,4,2,11,30,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59999,59999,5,1.719773e+09,1.719779e+09,1.719775e+09,3,30,16,1026.200000,3213,...,1.341996,3120.200,6856.000,22128.920,4425.784000,59999,0,0,3,0
60000,60000,2,1.719778e+09,1.719779e+09,1.719779e+09,23,30,13,403.500000,757,...,0.287587,2716.416,5610.840,8327.256,4163.628000,60000,0,0,0,0
60001,60001,1,1.719775e+09,1.719775e+09,1.719775e+09,19,30,10,1780.000000,1780,...,4.674124,380.820,380.820,380.820,380.820000,60001,0,0,1,0
60002,60002,2,1.719775e+09,1.719781e+09,1.719778e+09,10,31,16,2566.500000,4941,...,0.922378,5557.000,5778.000,11335.000,5667.500000,60002,0,0,1,0


In [130]:
df = pd.DataFrame()
columns = ['viewer_uid', 'event_timestamp_timezone_count',
           'event_timestamp_timezone_min', 'event_timestamp_timezone_max',
           'event_timestamp_timezone_mean', 'region_get_mode', 'label_get_mode',
           'category_get_mode', 'total_watchtime_mean', 'total_watchtime_max',
           'total_watchtime_min', 'total_watchtime_sum', 'device_get_mode',
           'author_id_get_mode', 'rutube_video_id_get_mode', 'percent_mean',
           'percent_min', 'percent_max', 'percent_sum', 'duration_min',
           'duration_max', 'duration_sum', 'duration_mean', 'morning_sum', 'day_sum', 'evening_sum', 'night_sum']

In [131]:
for i in columns:
    df[i] = aggregated[i]

In [132]:
df

,viewer_uid,event_timestamp_timezone_count,event_timestamp_timezone_min,event_timestamp_timezone_max,event_timestamp_timezone_mean,region_get_mode,label_get_mode,category_get_mode,total_watchtime_mean,total_watchtime_max,...,percent_max,percent_sum,duration_min,duration_max,duration_sum,duration_mean,morning_sum,day_sum,evening_sum,night_sum
0,0,9,1.717874e+09,1.718833e+09,1.718255e+09,52,30,17,1602.111111,4821,...,0.997214,4.338079,1780.480,5650.134,29170.703,3241.189222,0,3,2,3
1,1,1,1.717573e+09,1.717573e+09,1.717573e+09,78,30,17,92.000000,92,...,0.057440,0.057440,1601.680,1601.680,1601.680,1601.680000,1,0,0,0
2,2,2,1.717259e+09,1.717331e+09,1.717295e+09,14,30,17,1735.000000,2560,...,0.710736,0.963381,3601.898,3601.898,7203.796,3601.898000,0,1,1,0
3,3,4,1.717229e+09,1.719658e+09,1.718596e+09,11,40,5,1119.500000,1202,...,1.017071,3.951202,1100.075,1181.825,4532.333,1133.083250,1,2,0,0
4,4,52,1.717225e+09,1.719775e+09,1.718193e+09,14,35,1,1411.961538,4919,...,0.983639,27.364966,1034.529,7426.427,137341.970,2641.191731,2,11,30,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59999,59999,5,1.719773e+09,1.719779e+09,1.719775e+09,3,30,16,1026.200000,3213,...,0.925670,1.341996,3120.200,6856.000,22128.920,4425.784000,0,0,3,0
60000,60000,2,1.719778e+09,1.719779e+09,1.719779e+09,23,30,13,403.500000,757,...,0.278676,0.287587,2716.416,5610.840,8327.256,4163.628000,0,0,0,0
60001,60001,1,1.719775e+09,1.719775e+09,1.719775e+09,19,30,10,1780.000000,1780,...,4.674124,4.674124,380.820,380.820,380.820,380.820000,0,0,1,0
60002,60002,2,1.719775e+09,1.719781e+09,1.719778e+09,10,31,16,2566.500000,4941,...,0.889149,0.922378,5557.000,5778.000,11335.000,5667.500000,0,0,1,0


In [145]:
from catboost import CatBoostClassifier, CatBoostRegressor

sex_classifier = CatBoostClassifier()
sex_classifier.load_model('sex_model')

In [146]:
age_classifier = CatBoostClassifier()
age_classifier.load_model("age_model")

In [147]:
age_regressor = CatBoostRegressor()
age_regressor.load_model("int_age_model")

In [135]:
df

,viewer_uid,event_timestamp_timezone_count,event_timestamp_timezone_min,event_timestamp_timezone_max,event_timestamp_timezone_mean,region_get_mode,label_get_mode,category_get_mode,total_watchtime_mean,total_watchtime_max,...,percent_max,percent_sum,duration_min,duration_max,duration_sum,duration_mean,morning_sum,day_sum,evening_sum,night_sum
0,0,9,1.717874e+09,1.718833e+09,1.718255e+09,52,30,17,1602.111111,4821,...,0.997214,4.338079,1780.480,5650.134,29170.703,3241.189222,0,3,2,3
1,1,1,1.717573e+09,1.717573e+09,1.717573e+09,78,30,17,92.000000,92,...,0.057440,0.057440,1601.680,1601.680,1601.680,1601.680000,1,0,0,0
2,2,2,1.717259e+09,1.717331e+09,1.717295e+09,14,30,17,1735.000000,2560,...,0.710736,0.963381,3601.898,3601.898,7203.796,3601.898000,0,1,1,0
3,3,4,1.717229e+09,1.719658e+09,1.718596e+09,11,40,5,1119.500000,1202,...,1.017071,3.951202,1100.075,1181.825,4532.333,1133.083250,1,2,0,0
4,4,52,1.717225e+09,1.719775e+09,1.718193e+09,14,35,1,1411.961538,4919,...,0.983639,27.364966,1034.529,7426.427,137341.970,2641.191731,2,11,30,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59999,59999,5,1.719773e+09,1.719779e+09,1.719775e+09,3,30,16,1026.200000,3213,...,0.925670,1.341996,3120.200,6856.000,22128.920,4425.784000,0,0,3,0
60000,60000,2,1.719778e+09,1.719779e+09,1.719779e+09,23,30,13,403.500000,757,...,0.278676,0.287587,2716.416,5610.840,8327.256,4163.628000,0,0,0,0
60001,60001,1,1.719775e+09,1.719775e+09,1.719775e+09,19,30,10,1780.000000,1780,...,4.674124,4.674124,380.820,380.820,380.820,380.820000,0,0,1,0
60002,60002,2,1.719775e+09,1.719781e+09,1.719778e+09,10,31,16,2566.500000,4941,...,0.889149,0.922378,5557.000,5778.000,11335.000,5667.500000,0,0,1,0


In [136]:
sex_classes = sex_classifier.predict(df.drop(['viewer_uid'], axis=1))

In [137]:
age_classes = age_classifier.predict(aggregated.drop(['viewer_uid'], axis=1))
age_classes

array([[2],
       [2],
       [2],
       ...,
       [3],
       [1],
       [2]], dtype=int64)

In [148]:
int_ages = age_regressor.predict(aggregated.drop(['viewer_uid'], axis=1))
int_ages

array([35.68152548, 43.02488342, 32.05420673, ..., 34.82979698,
       33.8488478 , 32.53072412])

In [150]:
import numpy as np

int_ages = np.round(int_ages).astype(int)

In [151]:
df2 = pd.DataFrame()

df2['viewer_uid'] = df['viewer_uid']
df2['sex'] = sex_classes
df2['age_class'] = age_classes
df2['age'] = int_ages
df2

,viewer_uid,sex,age_class,age
0,0,1,2,36
1,1,1,2,43
2,2,1,2,32
3,3,1,1,35
4,4,0,1,19
...,...,...,...,...
59999,59999,0,1,35
60000,60000,0,2,32
60001,60001,1,3,35
60002,60002,1,1,34


In [152]:
df2['sex'] = df2['sex'].map({1:'male', 0:'female'})
df2

,viewer_uid,sex,age_class,age
0,0,male,2,36
1,1,male,2,43
2,2,male,2,32
3,3,male,1,35
4,4,female,1,19
...,...,...,...,...
59999,59999,female,1,35
60000,60000,female,2,32
60001,60001,male,3,35
60002,60002,male,1,34


In [153]:
df2.to_csv("submission.csv", index=False)